In [ ]:
#2点間のトラカン案分を行う。テストのため、案分対象は緯度経度で絞っている
#トラカンの位置にKDDIODデータが存在しないなら案分の対象としない。

In [1]:
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [2]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [3]:
dbname = "kddi_12_new"
stdir = "230313"

In [11]:
find = 0

#KDDI生データを読み込む
f = open(stdir+'/'+ dbname + 'dummy.csv', 'w')
f.write("times,mesh,tbran\n")

cur =connection.cursor()
sql = "SELECT kddi.times, kddi.mesh, kddi.tbran, pt.latitude, pt.longitude, pt.geom FROM "+dbname+" AS kddi "
sql += "INNER JOIN pointdata AS pt on kddi.mesh = cast(pt.grid_code AS BIGINT) "
sql += "WHERE pt.latitude < 33.822785 AND pt.longitude > 132.755106 AND pt.latitude > 33.790157 AND pt.longitude < 132.789578 "
sql += "ORDER BY kddi.times, kddi.mesh "
cur.execute(sql)
for row in cur:
    #print(row)
    latlngStr = str(row[4]) + " " + str(row[3])
    latlng = str(row[4]) + "," + str(row[3])
    andlatlng = " where pt.latitude > "+ str(float(row[3])-0.2) +" AND pt.latitude < "+ str(float(row[3])+0.2)
    andlatlng += " AND pt.longitude > "+ str(float(row[4])-0.2) +" AND pt.longitude < "+ str(float(row[4])+0.2) + " "
    #一番近いKDDI測定値の2点を導き出す
    #ただし、同じ拠点データは加算すること
    cur2 =connection.cursor()
    sql = "SELECT "
    sql += "trkn.gridcode, (ST_Distance('SRID=4612;POINT("+latlngStr+")'::GEOGRAPHY, pt.geom) / 125) AS st, "
    sql += "trkn.断面交通量 AS danmen, kddi.tbran, pt.latitude, pt.longitude "
    sql += "FROM trakan2204_z3out_15min AS trkn "
    sql += "INNER JOIN "+dbname+" AS kddi ON  trkn.gridcode = kddi.mesh AND  kddi.times = trkn.時刻 "
    sql += "INNER JOIN pointdata AS pt ON cast(pt.grid_code AS BIGINT) = trkn.gridcode "
    sql += "WHERE trkn.時刻 = '"+str(row[0])+"'"
    sql += "ORDER BY trkn.place <-> point("+latlng+")"
    sql += "LIMIT 10 "
    #print(sql)
    cur2.execute(sql)
    kddiDf = pd.DataFrame.from_records(cur2.fetchall(), columns=["gridcode","st","danmen","tbran","latitude","longitude"])
    #print(kddiDf)
    cur2.close()
    
    if len(kddiDf) < 2:
        print("check error " + str(len(kddiDf)) + " finds!!")
        continue
    
    mode = 0
    for i in range(len(kddiDf)):
        wktrkn = kddiDf.iloc[i]
        wklatlng = str(wktrkn["longitude"]) + " " + str(wktrkn["latitude"])
        if i == 0:
            chktrknA = wktrkn
            latlngA = str(wktrkn["longitude"]) + " " + str(wktrkn["latitude"])
            tbranA =  wktrkn["tbran"]
        if mode == 0:
            if latlngA == wklatlng:
                tbranA = tbranA + wktrkn["tbran"]
            else:
                mode = 1
                chktrknB = wktrkn
                tbranB =  wktrkn["tbran"]
                latlngB = str(wktrkn["longitude"]) + " " + str(wktrkn["latitude"])
        elif latlngB == wklatlng:
            tbranB = tbranB + wktrkn["tbran"]
        else:
            break

    #一番近いKDDI測定値の2点を導き出す
    cur3 =connection.cursor()
    sql = "SELECT "
    sql += "(ST_Distance('SRID=4612;POINT("+latlngA+")'::GEOGRAPHY, 'SRID=4612;POINT("+latlngB+")'::GEOGRAPHY) / 125) AS st  "
    sql += "FROM kddi_00_new AS kddi "
    cur3.execute(sql)
    chkAB = 0

    for row3 in cur3:
        chkAB = row3[0]
    cur3.close()

    if chkAB == 0:
        print(latlngA + "*" + latlngB + "check A-B is zero!!")
        continue

    #片方によっている
    #print(str(chkAB))
    if chkAB <  chktrknA["st"]:
        #Aのみで案分
        chg = chktrknA["danmen"] * (row[2] / tbranA)
    elif chkAB <  chktrknB["st"]:
        #Bのみで案分
        chg = chktrknB["danmen"] * (row[2] / tbranB) 
    else:
        if chktrknA["danmen"] > chktrknB["danmen"]:
            wk = chktrknA
            chktrknA = chktrknB
            chktrknB = wk
        x = (chktrknB["danmen"] - chktrknA["danmen"]) / (chktrknA["st"] + chktrknB["st"])
        chg = (chktrknA["danmen"] + (x * chktrknA["st"])) *  (row[2] / tbranA) 
        if chg > 2000:
            print("("+str(chktrknA["danmen"]) + "+(" + str(x) + "*" + str(chktrknA["st"]) + "))*" + str(row[2]) + "/" + str(chktrknA["tbran"]))
            print(str(chg))
    f.write(str(row[0]) + "," + str(row[1]) + "," + str(chg) + "\n")
    #print(str(row[0]) + "," + str(row[1]) + "," + str(chg))
    find = find + 1

    if find % 1000 == 0:
        print(str(find))
        break
f.close()   
cur.close()
    
    

1000


In [12]:
connection.close()

In [ ]:
#CreateGeojsonFile